In [8]:
import parse_python3
import ast
import astor

In [1]:
filenm = "new_files/fingerping.py"
astree_json = parse_python3.parse_file(filenm)

In [3]:
def read_file_to_string(filename):
    f = open(filename, 'rt')
    s = f.read()
    f.close()
    return s

code = read_file_to_string(filenm)
print("Length of code:", len(code))

Length of code: 7689


In [4]:
print(code)

#!/usr/bin/python
#
# fingerping: A PNG library fingerprinting tool.
# 
# @author:Dominique Bongard
# 
#
# Code is licensed under -- Apache License 2.0 http://www.apache.org/licenses/
#

from collections import namedtuple
from operator import *
from struct import *
import os.path
import sys
import zlib
import binascii
import itertools
from xpng import *

# A named tuple representing a fingerprint test
# name = name of the test
# filename = filename of the png file to test
# function = function to call to execute the test
# description = description of the test
Test = namedtuple("test", "name filename function description")

# A named tuple representing the fingerprint of a PNG library
# name = name of the language / library / tool
# description = description of the library (e.g. version number)
# results = a dictionary of test fingerprint test results
Fingerprint = namedtuple("fingerprint","name description results")


###############################################
#
# Fingerprinting 

In [9]:
astree = ast.parse(code, filenm)
back_translate_code = astor.to_source(astree)
print("Length of code:", len(back_translate_code))

Length of code: 4639


In [12]:
class CommentProcessor:
    def __init__(self):
        self.comments = {}
    
    def parse_comments(self, code):
        for lineno, line in enumerate(code.split("\n")):
            if "#" in line:
                comment = line[line.find("#")+1:]
                line = line[:line.find("#")]
                quotes1 = line.count('"')
                quotes2 = line.count("'")
                comment = re.sub('\t+', ' ', comment.strip().replace("#", ""))
                comment = re.sub(' +', ' ', comment)
                if len(comment) and quotes1 % 2 == 0 and quotes2 % 2 == 0:
                    self.comments[lineno] = comment

In [14]:
import re

cp = CommentProcessor()
cp.parse_comments(code)

In [16]:
cp.comments

{0: '!/usr/bin/python',
 2: 'fingerping: A PNG library fingerprinting tool.',
 4: '@author:Dominique Bongard',
 7: 'Code is licensed under -- Apache License 2.0 http://www.apache.org/licenses/',
 20: 'A named tuple representing a fingerprint test',
 21: 'name = name of the test',
 22: 'filename = filename of the png file to test',
 23: 'function = function to call to execute the test',
 24: 'description = description of the test',
 27: 'A named tuple representing the fingerprint of a PNG library',
 28: 'name = name of the language / library / tool',
 29: 'description = description of the library (e.g. version number)',
 30: 'results = a dictionary of test fingerprint test results',
 36: 'Fingerprinting functions',
 41: 'The most simple fingerprinting function',
 43: 'Returns 0 if the image is absent or empty (meaning the target failed to decode the input image)',
 44: 'Returns 10 if the image looks valid at least in surface',
 45: 'Returns between 1 and 9 if the image is corrupt',
 49: